# Đồ Án Cuối Kì Môn Khoa Học Dữ Liệu

**Giới thiệu thành viên**

- *Họ tên:* **Trần Ngọc Tịnh**       
  *MSSV:* **18120597**
  
  
  
  
- *Họ tên:* **Nguyễn Ngọc Năng Toàn**       
  *MSSV:* **18120600**

## Import 

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns # seaborn là thư viện được xây trên matplotlib, giúp việc visualization đỡ khổ hơn
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.neural_network import MLPClassifier
from sklearn import set_config
set_config(display='diagram') # Để trực quan hóa pipeline

# You can also import other things ...
# YOUR CODE HERE (OPTION)

## Đọc dữ liệu từ file csv

In [2]:
data_df = pd.read_csv('data.csv', 
                      index_col=0) # Cho cột index là cột 0 (PassengerId)
data_df.head()

,confirmed,recovered,deaths,population,sq_km_area,life_expectancy,elevation_in_meters,continent,abbreviation,location,iso,capital_city,lat,long,updated
Country,,,,,,,,,,,,,,,
Afghanistan,51526,41727,2191,35530081.0,652090.0,45.9,NaN,Asia,AF,Southern and Central Asia,4.0,Kabul,33.93911,67.709953,2021/01/01 15:22:42+00
Albania,58316,33634,1181,2930187.0,28748.0,71.6,NaN,Europe,AL,Southern Europe,8.0,Tirana,41.15330,20.168300,2021/01/01 15:22:42+00
Algeria,99610,67127,2756,41318142.0,2381741.0,69.7,800,Africa,DZ,Northern Africa,12.0,Alger,28.03390,1.659600,2021/01/01 15:22:42+00
Andorra,8049,7432,84,76965.0,468.0,83.5,"1,996",Europe,AD,Southern Europe,20.0,Andorra la Vella,42.50630,1.521800,2021/01/01 15:22:42+00
Angola,17553,11044,405,29784193.0,1246700.0,38.3,"1,112",Africa,AO,Central Africa,24.0,Luanda,-11.20270,17.873900,2021/01/01 15:22:42+00


In [3]:
data_df.index.duplicated().sum()

0

In [4]:
data_df['deaths'].dtype

dtype('int64')

In [5]:
data_df['deaths'].isna().sum()

0

In [6]:
data_df['deaths'].value_counts(normalize=True) * 100

0       7.558140
63      1.744186
1468    1.162791
7576    1.162791
104     1.162791
          ...   
1356    0.581395
335     0.581395
2641    0.581395
83      0.581395
76      0.581395
Name: deaths, Length: 150, dtype: float64

In [7]:
y_sr = data_df["deaths"] # sr là viết tắt của series
X_df = data_df.drop("deaths", axis=1)

In [9]:
train_X, test_X, train_y, test_y = train_test_split(X_df, y_sr, test_size=0.3, random_state=0)

In [51]:
unused_cols = ['abbreviation', 'capital_city', 'updated','location']
X_temp=X_df.drop(unused_cols,axis=1)
X_temp=X_temp.astype({'confirmed':'float64','recovered':'float64'})
X_temp['elevation_in_meters'] = pd.to_numeric(X_temp['elevation_in_meters'],errors='coerce')
X_temp.head()

,confirmed,recovered,population,sq_km_area,life_expectancy,elevation_in_meters,continent,iso,lat,long
Country,,,,,,,,,,
Afghanistan,51526.0,41727.0,35530081.0,652090.0,45.9,NaN,Asia,4.0,33.93911,67.709953
Albania,58316.0,33634.0,2930187.0,28748.0,71.6,NaN,Europe,8.0,41.15330,20.168300
Algeria,99610.0,67127.0,41318142.0,2381741.0,69.7,800.0,Africa,12.0,28.03390,1.659600
Andorra,8049.0,7432.0,76965.0,468.0,83.5,NaN,Europe,20.0,42.50630,1.521800
Angola,17553.0,11044.0,29784193.0,1246700.0,38.3,NaN,Africa,24.0,-11.20270,17.873900


In [45]:
X_temp.dtypes

confirmed              float64
recovered              float64
population             float64
sq_km_area             float64
life_expectancy        float64
elevation_in_meters    float64
continent               object
iso                    float64
lat                    float64
long                   float64
dtype: object

In [49]:
# YOUR CODE HERE
nume_cols=['confirmed','recovered','population','sq_km_area','life_expectancy','iso','lat','long','elevation_in_meters']
cate_cols=['continent']
mean = SimpleImputer(strategy='mean')
mode = SimpleImputer(strategy='most_frequent') 
mode_encode = make_pipeline(mode, OneHotEncoder(handle_unknown='ignore'))

coltrans = make_column_transformer((mean, nume_cols),
                                (mode_encode,cate_cols),
                                   remainder='drop')

preprocess_pipeline = make_pipeline(coltrans,StandardScaler())

preprocessed_train_X=preprocess_pipeline.fit_transform(X_temp)

preprocess_pipeline

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('simpleimputer',
                                                  SimpleImputer(),
                                                  ['confirmed', 'recovered',
                                                   'population', 'sq_km_area',
                                                   'life_expectancy', 'iso',
                                                   'lat', 'long',
                                                   'elevation_in_meters']),
                                                 ('pipeline',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehotencoder',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['continent'])])),
                ('standardscaler', StandardScaler())])

In [50]:
preprocessed_train_X

array([[-0.27492024, -0.22218113, -0.02888944, ..., -0.38297084,
        -0.20597146, -0.27386128],
       [-0.26853297, -0.23068894, -0.24573853, ..., -0.38297084,
        -0.20597146, -0.27386128],
       [-0.22968818, -0.19547924,  0.00961179, ..., -0.38297084,
        -0.20597146, -0.27386128],
       ...,
       [-0.32141565, -0.26458142, -0.07731254, ..., -0.38297084,
        -0.20597146, -0.27386128],
       [-0.30363852, -0.24631165, -0.15152233, ..., -0.38297084,
        -0.20597146, -0.27386128],
       [-0.31034563, -0.25422024, -0.15527547, ..., -0.38297084,
        -0.20597146, -0.27386128]])